In [54]:
import pandas as pd
import numpy as np 
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
from datetime import datetime
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from imblearn.over_sampling import SMOTE, ADASYN
sm = SMOTE(random_state = 2)
ada = ADASYN(random_state = 42, sampling_strategy='minority')
import tensorflow as tf
#from keras.models import Sequential
#from keras.layers import Dense
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.ensemble import BalancedBaggingClassifier
from imblearn.under_sampling import RandomUnderSampler

In [7]:
def g11(val): 
    """
    Records whether or not a number is greater than 11. 
    """
    if val <= 11: 
        return 0
    else: 
        return 1

In [8]:
df = pd.read_csv('fservice.csv')
df['Eleven'] = df['ElapsedDays'].apply(g11, 0)

values = {'MobileOS': "NotMobile", 'CreatedByUserOrganization': 'NotSpecified'}

df = df.fillna(value=values)

In [9]:
df['Just Date'] = df['Just Date'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d'))

In [10]:
df['day_of_week'] = df['Just Date'].dt.day_name()

In [11]:
df['Month'] = df['Just Date'].dt.month_name()

In [12]:
#Columns 

#c = ['Anonymous','CreatedByUserOrganization', 'RequestType','MobileOS','RequestSource', 'Month'] #BEST

c = ['Anonymous','CreatedByUserOrganization', 'RequestType','MobileOS','RequestSource', 'Month']


n = ['Latitude', 'Longitude']

In [13]:
#dfm = df[df['Eleven'] == 1]

In [14]:
#dfl = df[df['Eleven'] == 0].sample(dfm.shape[0])

In [15]:
#dfn = dfl.append(dfm)
dfn = df.filter(items = c + n  + ['Eleven'])
dfn = dfn.dropna()

In [16]:
#Separate data into explanatory and response variables
XCAT = dfn.filter(items = c).values
XNUM = dfn.filter(items = n).values

y = np.array(dfn['Eleven'].values, dtype=np.float)

#Encode cateogrical data and merge with numerical data
labelencoder_X = LabelEncoder()
for num in range(len(c)):
    XCAT[:, num] = labelencoder_X.fit_transform(XCAT[:, num])

onehotencoder = OneHotEncoder()
XCAT = onehotencoder.fit_transform(XCAT).toarray()

X = np.concatenate((XCAT, XNUM), axis=1)

In [17]:
dfn['RequestType'].unique()

array(['Other', 'Graffiti Removal', 'Bulky Items', 'Dead Animal Removal',
       'Metal/Household Appliances', 'Single Streetlight Issue',
       'Illegal Dumping Pickup', 'Feedback', 'Multiple Streetlight Issue',
       'Report Water Waste', 'Homeless Encampment', 'Electronic Waste'],
      dtype=object)

In [19]:
#Split data 

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    #stratify=1, 
                                                    test_size=0.2, random_state=0)

In [20]:
rus = RandomUnderSampler(random_state=0)
X_train_undersampled, y_train_undersampled = rus.fit_resample(X_train, y_train)

In [65]:
print("Percent minority class for y_train and y_train_undersampled:")
print(np.unique(y_train, return_counts=True)[1][1] / np.unique(y_train, return_counts=True)[1][0])
print(np.unique(y_train_undersampled, return_counts=True)[1][1] / np.unique(y_train_undersampled, return_counts=True)[1][0])
print("Percent minority class for y_test:")
print(np.unique(y_test, return_counts=True)[1][1] / np.unique(y_test, return_counts=True)[1][0])

Percent minority class for y_train and y_train_undersampled:
0.05412102232956836
1.0
Percent minority class for y_test:
0.05452606775034895


In [84]:
rf = RandomForestClassifier(n_estimators = 200, max_depth = 20, verbose=2, n_jobs=-1)  #Current best
et = ExtraTreesClassifier(n_estimators=10, max_depth=20, min_samples_split=2, random_state=0)
ab = AdaBoostClassifier(n_estimators=20)
gb = GradientBoostingClassifier(n_estimators=200, learning_rate=0.1, max_depth=10, verbose=2, random_state=0, n_iter_no_change=1)
hgb = HistGradientBoostingClassifier(loss='binary_crossentropy', max_iter=10000, learning_rate=0.05, max_depth=30, verbose=2, random_state=0)

brf = BalancedRandomForestClassifier(n_estimators=50, max_depth=20, random_state=0, verbose=2, n_jobs=-1)
bbc = BalancedBaggingClassifier(base_estimator=RandomForestClassifier(n_estimators=50, max_depth=20, verbose=2), sampling_strategy='auto', replacement=False, random_state=0, verbose=2, n_jobs=3)

In [85]:
classifier = hgb
classifier.fit(X_train_undersampled, y_train_undersampled)


##TEST SET##
y_tpred = classifier.predict(X_test)

#Print Test results
print("Test Set Results")
print(metrics.classification_report(y_test, y_tpred, digits=3))

Binning 0.124 GB of training data: 0.805 s
Binning 0.014 GB of validation data: 0.012 s
Fitting gradient boosted rounds:
[1/10000] 1 tree, 31 leaves, max depth = 16, train loss: 0.66946, val loss: 0.66948, in 0.105s
[2/10000] 1 tree, 31 leaves, max depth = 16, train loss: 0.64800, val loss: 0.64805, in 0.092s
[3/10000] 1 tree, 31 leaves, max depth = 16, train loss: 0.62856, val loss: 0.62861, in 0.068s
[4/10000] 1 tree, 31 leaves, max depth = 15, train loss: 0.61084, val loss: 0.61092, in 0.073s
[5/10000] 1 tree, 31 leaves, max depth = 16, train loss: 0.59466, val loss: 0.59471, in 0.061s
[6/10000] 1 tree, 31 leaves, max depth = 16, train loss: 0.57989, val loss: 0.57991, in 0.060s
[7/10000] 1 tree, 31 leaves, max depth = 15, train loss: 0.56639, val loss: 0.56644, in 0.071s
[8/10000] 1 tree, 31 leaves, max depth = 15, train loss: 0.55398, val loss: 0.55404, in 0.063s
[9/10000] 1 tree, 31 leaves, max depth = 16, train loss: 0.54257, val loss: 0.54262, in 0.062s
[10/10000] 1 tree, 31 le

[86/10000] 1 tree, 31 leaves, max depth = 11, train loss: 0.38871, val loss: 0.39069, in 0.075s
[87/10000] 1 tree, 31 leaves, max depth = 10, train loss: 0.38849, val loss: 0.39049, in 0.083s
[88/10000] 1 tree, 31 leaves, max depth = 12, train loss: 0.38822, val loss: 0.39022, in 0.075s
[89/10000] 1 tree, 31 leaves, max depth = 14, train loss: 0.38802, val loss: 0.39004, in 0.317s
[90/10000] 1 tree, 31 leaves, max depth = 11, train loss: 0.38784, val loss: 0.38991, in 0.254s
[91/10000] 1 tree, 31 leaves, max depth = 20, train loss: 0.38767, val loss: 0.38976, in 0.210s
[92/10000] 1 tree, 31 leaves, max depth = 13, train loss: 0.38738, val loss: 0.38950, in 0.102s
[93/10000] 1 tree, 31 leaves, max depth = 17, train loss: 0.38720, val loss: 0.38933, in 0.097s
[94/10000] 1 tree, 31 leaves, max depth = 12, train loss: 0.38704, val loss: 0.38919, in 0.107s
[95/10000] 1 tree, 31 leaves, max depth = 18, train loss: 0.38689, val loss: 0.38906, in 0.137s
[96/10000] 1 tree, 31 leaves, max depth 

[171/10000] 1 tree, 31 leaves, max depth = 9, train loss: 0.37788, val loss: 0.38186, in 0.053s
[172/10000] 1 tree, 31 leaves, max depth = 11, train loss: 0.37779, val loss: 0.38178, in 0.053s
[173/10000] 1 tree, 31 leaves, max depth = 12, train loss: 0.37771, val loss: 0.38172, in 0.073s
[174/10000] 1 tree, 31 leaves, max depth = 11, train loss: 0.37756, val loss: 0.38161, in 0.050s
[175/10000] 1 tree, 31 leaves, max depth = 10, train loss: 0.37747, val loss: 0.38155, in 0.046s
[176/10000] 1 tree, 31 leaves, max depth = 11, train loss: 0.37742, val loss: 0.38153, in 0.051s
[177/10000] 1 tree, 31 leaves, max depth = 11, train loss: 0.37725, val loss: 0.38139, in 0.057s
[178/10000] 1 tree, 31 leaves, max depth = 10, train loss: 0.37719, val loss: 0.38134, in 0.054s
[179/10000] 1 tree, 31 leaves, max depth = 14, train loss: 0.37713, val loss: 0.38132, in 0.064s
[180/10000] 1 tree, 31 leaves, max depth = 12, train loss: 0.37708, val loss: 0.38131, in 0.046s
[181/10000] 1 tree, 31 leaves, 

[256/10000] 1 tree, 31 leaves, max depth = 15, train loss: 0.37129, val loss: 0.37760, in 0.047s
[257/10000] 1 tree, 31 leaves, max depth = 8, train loss: 0.37122, val loss: 0.37756, in 0.054s
[258/10000] 1 tree, 31 leaves, max depth = 9, train loss: 0.37118, val loss: 0.37753, in 0.043s
[259/10000] 1 tree, 31 leaves, max depth = 12, train loss: 0.37114, val loss: 0.37751, in 0.043s
[260/10000] 1 tree, 31 leaves, max depth = 14, train loss: 0.37111, val loss: 0.37750, in 0.041s
[261/10000] 1 tree, 31 leaves, max depth = 13, train loss: 0.37108, val loss: 0.37750, in 0.043s
[262/10000] 1 tree, 31 leaves, max depth = 15, train loss: 0.37104, val loss: 0.37748, in 0.041s
[263/10000] 1 tree, 31 leaves, max depth = 10, train loss: 0.37097, val loss: 0.37744, in 0.049s
[264/10000] 1 tree, 31 leaves, max depth = 11, train loss: 0.37094, val loss: 0.37743, in 0.046s
[265/10000] 1 tree, 31 leaves, max depth = 9, train loss: 0.37090, val loss: 0.37742, in 0.055s
[266/10000] 1 tree, 31 leaves, ma

[341/10000] 1 tree, 31 leaves, max depth = 12, train loss: 0.36657, val loss: 0.37501, in 0.048s
[342/10000] 1 tree, 31 leaves, max depth = 10, train loss: 0.36653, val loss: 0.37499, in 0.046s
[343/10000] 1 tree, 31 leaves, max depth = 12, train loss: 0.36644, val loss: 0.37496, in 0.049s
[344/10000] 1 tree, 31 leaves, max depth = 8, train loss: 0.36637, val loss: 0.37492, in 0.043s
[345/10000] 1 tree, 31 leaves, max depth = 9, train loss: 0.36628, val loss: 0.37484, in 0.050s
[346/10000] 1 tree, 31 leaves, max depth = 12, train loss: 0.36615, val loss: 0.37473, in 0.049s
[347/10000] 1 tree, 31 leaves, max depth = 10, train loss: 0.36613, val loss: 0.37473, in 0.040s
[348/10000] 1 tree, 31 leaves, max depth = 10, train loss: 0.36607, val loss: 0.37470, in 0.050s
[349/10000] 1 tree, 31 leaves, max depth = 10, train loss: 0.36604, val loss: 0.37469, in 0.042s
[350/10000] 1 tree, 31 leaves, max depth = 12, train loss: 0.36601, val loss: 0.37468, in 0.046s
[351/10000] 1 tree, 31 leaves, m

...

[1609/10000] 1 tree, 31 leaves, max depth = 14, train loss: 0.32618, val loss: 0.35727, in 0.048s
[1610/10000] 1 tree, 31 leaves, max depth = 13, train loss: 0.32617, val loss: 0.35727, in 0.048s
[1611/10000] 1 tree, 31 leaves, max depth = 17, train loss: 0.32616, val loss: 0.35727, in 0.052s
[1612/10000] 1 tree, 31 leaves, max depth = 13, train loss: 0.32615, val loss: 0.35727, in 0.049s
[1613/10000] 1 tree, 31 leaves, max depth = 14, train loss: 0.32612, val loss: 0.35725, in 0.053s
[1614/10000] 1 tree, 31 leaves, max depth = 12, train loss: 0.32610, val loss: 0.35724, in 0.049s
[1615/10000] 1 tree, 31 leaves, max depth = 11, train loss: 0.32609, val loss: 0.35724, in 0.054s
[1616/10000] 1 tree, 31 leaves, max depth = 14, train loss: 0.32608, val loss: 0.35725, in 0.080s
[1617/10000] 1 tree, 31 leaves, max depth = 12, train loss: 0.32605, val loss: 0.35724, in 0.069s
[1618/10000] 1 tree, 31 leaves, max depth = 11, train loss: 0.32603, val loss: 0.35724, in 0.053s
[1619/10000] 1 tree,

[1693/10000] 1 tree, 31 leaves, max depth = 11, train loss: 0.32449, val loss: 0.35678, in 0.055s
[1694/10000] 1 tree, 31 leaves, max depth = 17, train loss: 0.32447, val loss: 0.35677, in 0.057s
[1695/10000] 1 tree, 31 leaves, max depth = 8, train loss: 0.32444, val loss: 0.35675, in 0.052s
[1696/10000] 1 tree, 31 leaves, max depth = 9, train loss: 0.32441, val loss: 0.35676, in 0.054s
[1697/10000] 1 tree, 31 leaves, max depth = 14, train loss: 0.32439, val loss: 0.35674, in 0.055s
[1698/10000] 1 tree, 31 leaves, max depth = 12, train loss: 0.32436, val loss: 0.35673, in 0.056s
[1699/10000] 1 tree, 31 leaves, max depth = 14, train loss: 0.32433, val loss: 0.35672, in 0.054s
[1700/10000] 1 tree, 31 leaves, max depth = 11, train loss: 0.32432, val loss: 0.35672, in 0.052s
[1701/10000] 1 tree, 31 leaves, max depth = 11, train loss: 0.32430, val loss: 0.35671, in 0.062s
[1702/10000] 1 tree, 31 leaves, max depth = 12, train loss: 0.32428, val loss: 0.35670, in 0.064s
[1703/10000] 1 tree, 3

[1777/10000] 1 tree, 31 leaves, max depth = 10, train loss: 0.32260, val loss: 0.35606, in 0.059s
[1778/10000] 1 tree, 31 leaves, max depth = 15, train loss: 0.32257, val loss: 0.35605, in 0.050s
[1779/10000] 1 tree, 31 leaves, max depth = 14, train loss: 0.32256, val loss: 0.35604, in 0.045s
[1780/10000] 1 tree, 31 leaves, max depth = 13, train loss: 0.32254, val loss: 0.35604, in 0.040s
[1781/10000] 1 tree, 31 leaves, max depth = 10, train loss: 0.32253, val loss: 0.35604, in 0.054s
[1782/10000] 1 tree, 31 leaves, max depth = 10, train loss: 0.32251, val loss: 0.35604, in 0.045s
[1783/10000] 1 tree, 31 leaves, max depth = 14, train loss: 0.32249, val loss: 0.35604, in 0.047s
[1784/10000] 1 tree, 31 leaves, max depth = 10, train loss: 0.32245, val loss: 0.35602, in 0.090s
[1785/10000] 1 tree, 31 leaves, max depth = 12, train loss: 0.32243, val loss: 0.35601, in 0.055s
[1786/10000] 1 tree, 31 leaves, max depth = 9, train loss: 0.32241, val loss: 0.35602, in 0.047s
[1787/10000] 1 tree, 

[1861/10000] 1 tree, 31 leaves, max depth = 12, train loss: 0.32089, val loss: 0.35557, in 0.056s
[1862/10000] 1 tree, 31 leaves, max depth = 11, train loss: 0.32086, val loss: 0.35556, in 0.052s
[1863/10000] 1 tree, 31 leaves, max depth = 15, train loss: 0.32085, val loss: 0.35556, in 0.052s
[1864/10000] 1 tree, 31 leaves, max depth = 12, train loss: 0.32085, val loss: 0.35555, in 0.046s
[1865/10000] 1 tree, 31 leaves, max depth = 17, train loss: 0.32082, val loss: 0.35554, in 0.057s
[1866/10000] 1 tree, 31 leaves, max depth = 11, train loss: 0.32081, val loss: 0.35553, in 0.044s
[1867/10000] 1 tree, 31 leaves, max depth = 12, train loss: 0.32078, val loss: 0.35554, in 0.048s
[1868/10000] 1 tree, 31 leaves, max depth = 14, train loss: 0.32075, val loss: 0.35552, in 0.046s
[1869/10000] 1 tree, 31 leaves, max depth = 11, train loss: 0.32073, val loss: 0.35553, in 0.053s
[1870/10000] 1 tree, 31 leaves, max depth = 12, train loss: 0.32072, val loss: 0.35553, in 0.046s
[1871/10000] 1 tree,

[1945/10000] 1 tree, 31 leaves, max depth = 11, train loss: 0.31910, val loss: 0.35506, in 0.074s
[1946/10000] 1 tree, 31 leaves, max depth = 9, train loss: 0.31908, val loss: 0.35506, in 0.080s
[1947/10000] 1 tree, 31 leaves, max depth = 17, train loss: 0.31907, val loss: 0.35506, in 0.102s
[1948/10000] 1 tree, 31 leaves, max depth = 13, train loss: 0.31904, val loss: 0.35504, in 0.083s
[1949/10000] 1 tree, 31 leaves, max depth = 11, train loss: 0.31902, val loss: 0.35503, in 0.063s
[1950/10000] 1 tree, 31 leaves, max depth = 12, train loss: 0.31899, val loss: 0.35502, in 0.062s
[1951/10000] 1 tree, 31 leaves, max depth = 14, train loss: 0.31897, val loss: 0.35502, in 0.063s
[1952/10000] 1 tree, 31 leaves, max depth = 13, train loss: 0.31894, val loss: 0.35503, in 0.061s
[1953/10000] 1 tree, 31 leaves, max depth = 10, train loss: 0.31889, val loss: 0.35498, in 0.094s
[1954/10000] 1 tree, 31 leaves, max depth = 13, train loss: 0.31887, val loss: 0.35497, in 0.057s
[1955/10000] 1 tree, 

precision    recall  f1-score   support

         0.0      0.991     0.864     0.923    643362
         1.0      0.254     0.848     0.391     35080

    accuracy                          0.863    678442
   macro avg      0.622     0.856     0.657    678442
weighted avg      0.952     0.863     0.896    678442


In [25]:
df.columns

Index(['SRNumber', 'CreatedDate', 'UpdatedDate', 'ActionTaken', 'Owner',
       'RequestType', 'Status', 'RequestSource', 'MobileOS', 'Anonymous',
       'AssignTo', 'ServiceDate', 'ClosedDate', 'AddressVerified',
       'ApproximateAddress', 'Address', 'HouseNumber', 'Direction',
       'StreetName', 'Suffix', 'ZipCode', 'Latitude', 'Longitude', 'Location',
       'TBMPage', 'TBMColumn', 'TBMRow', 'APC', 'CD', 'CDMember', 'NC',
       'NCName', 'PolicePrecinct', 'CreatedByUserOrganization', 'ElapsedTime',
       'ElapsedDays', 'Just Date', 'Eleven', 'day_of_week', 'Month'],
      dtype='object')

In [26]:
dfa = dfl.append(dfm)

In [27]:
dfa[dfa['CreatedByUserOrganization'] == "ITA"]['Eleven'].value_counts()

1    11703
0     5898
Name: Eleven, dtype: int64

In [28]:
dfa['CreatedByUserOrganization'].unique()

array(['BOS', 'Self Service', 'NotSpecified', 'ITA', 'OCB',
       'Proactive Insert', 'Self Service_SAN', 'BSS', 'BSL',
       'BOS Franchise', "Council's Office", 'LAAS', 'FIMS', 'LASAN'],
      dtype=object)

In [29]:
c = ['Anonymous','CreatedByUserOrganization', 'RequestType','MobileOS','RequestSource', 'Month']